https://f1000research.com/articles/6-2055/v2

> #### Resolution of the DNA methylation state of single CpG dyads using in silico strand annealing and WGBS data
Nature protocol | https://www.nature.com/articles/s41596-018-0090-x

> - Whole-genome bisulfite sequencing (WGBS) has been widely used to quantify cytosine DNA methylation frequency in an expanding array of cell and tissue types. Because of the denaturing conditions used, this method ultimately leads to the measurement of methylation frequencies at single cytosines. 
> - Hence, the methylation frequency of CpG dyads (two complementary CG dinucleotides) can be only indirectly inferred by overlaying the methylation frequency of two cytosines measured independently. 
> - Furthermore, hemi-methylated CpGs (hemiCpGs) have not been previously analyzed in WGBS studies. We recently developed in silico strand annealing (iSA), a bioinformatics method applicable to WGBS data, to resolve the methylation status of CpG dyads into __unmethylated__, __hemi-methylated__, and __methylated__. 
> - HemiCpGs account for 4–20% of the DNA methylome in different cell types, and some can be inherited across cell divisions, suggesting a role as a stable epigenetic mark. Therefore, it is important to resolve hemiCpGs from fully methylated CpGs in WGBS studies. This protocol describes step-by-step commands to accomplish this task, including dividing alignments by strand, pairing alignments between strands, and extracting single-fragment methylation calls. 
> - The versatility of iSA enables its application downstream of other WGBS-related methods such as nasBS-seq (nascent DNA bisulfite sequencing), ChIP-BSseq (ChIP followed by bisulfite sequencing), TAB-seq, oxBS-seq, and fCAB-seq. iSA is also tunable for analyzing the methylation status of cytosines in any sequence context. 
> - We exemplify this flexibility by uncovering the single-fragment non-CpG methylome. 
> - This protocol provides enough details for users with little experience in bioinformatic analysis and takes 2–7 h.

> #### Extended-representation bisulfite sequencing of gene regulatory elements in multiplexed samples and single cells
https://www.nature.com/articles/s41587-021-00910-x

<!-- > - Before alignment, primer dimers were filtered using Cutadapt version 2.7 and the following parameters: `--discard -a GCTCTTCCGATCT`. Short read pairs were trimmed using Trim Galore version 0.6.5 and the following parameters: `--paired --illumina --nextseq 20`. 
> - High-quality sequencing reads were then aligned to an in silico bisulfite-converted reference genome (hg38 and mm10) using methylCtools version 1.0.0 (https://github.com/hovestadt/methylCtools, ref. 49) and bwa mem version 0.7.17. 
> - Sorted alignments were further processed to only maintain uniquely mapped read pairs with a mapping score ≥1, that were mapping to an MspI cut site and that had an insert size between 20 bp and 600 bp. Putative PCR duplicates were removed by considering the outer mapping position of both paired-end reads (read 2 being located at the MspI cut site and read 1 being located at variable positions), as well as the random hexamer sequence that was trimmed before alignment and functions as a UMI. For library complexity analysis, alignments were downsampled before this step. 
> - We note that multiple random hexamer priming events during the second-strand synthesis step might lead to additional sequencing reads from the same original fragment that cannot be identified using this approach. DNA methylation calling was performed using methylCtools bcall and the --trimPE parameter. Detailed quality metrics for each library are provided in Supplementary Table 1. DNA methylation values were deposited in the GEO (GSE149954) for all samples reported in this study.


 -->
 
 This is simply called RRBS protocol :)

# SRA Data

my `sra` env

`fastq-dump`

https://www.biostars.org/p/222122/


<!--     fastq-dump --outdir fastq --gzip --split-3 sra/SRR11711273.sra

    Read 17522209 spots for sra/SRR11711273.sra
    Written 17522209 spots for sra/SRR11711273.sra


___

    parallel-fastq-dump --threads 12 --outdir fastq/ --split-files --gzip sra/SRR11711272.sra -->
    

<!--     fasterq-dump --split-files --include-technical --threads 1 
    --temp . --outfile HL60_10ng_dmso.fastq --progress SRR11711272


    fasterq-dump --split-files --include-technical --threads 1 
    --temp . --outfile HL60_10ng_decitabine.fastq --progress SRR11711273 -->


    wget ftp.sra.ebi.ac.uk/vol1/fastq/SRR117/072/SRR11711272/SRR11711272_1.fastq.gz
    wget ftp.sra.ebi.ac.uk/vol1/fastq/SRR117/072/SRR11711272/SRR11711272_2.fastq.gz
    
    wget ftp.sra.ebi.ac.uk/vol1/fastq/SRR117/073/SRR11711273/SRR11711273_1.fastq.gz
    wget ftp.sra.ebi.ac.uk/vol1/fastq/SRR117/073/SRR11711273/SRR11711273_2.fastq.gz

In [3]:
mkdir fastq

In [4]:
!mv -v *fastq.gz fastq/

‘SRR11711272_1.fastq.gz’ -> ‘fastq/SRR11711272_1.fastq.gz’
‘SRR11711272_2.fastq.gz’ -> ‘fastq/SRR11711272_2.fastq.gz’
‘SRR11711273_1.fastq.gz’ -> ‘fastq/SRR11711273_1.fastq.gz’
‘SRR11711273_2.fastq.gz’ -> ‘fastq/SRR11711273_2.fastq.gz’


# Reading and annotating the methylation counts

## Trimming

<!-- > Before alignment, primer dimers were filtered using Cutadapt version 2.7 and the following parameters: `--discard -a GCTCTTCCGATCT`. Short read pairs were trimmed using Trim Galore version 0.6.5 and the following parameters: `--paired --illumina --nextseq 20`. -->

In [7]:
!mkdir -p logs/
!mkdir -p logs/trim_galore/

In [8]:
%%bash
for fq1 in fastq/*1.fastq.gz; do
    fq2=${fq1/1.fastq/2.fastq}
    b=`basename $fq1`
    log_file=${b/_1.fastq.gz/.log}
    cm="trim_galore --core 3 --paired --rrbs -o fastq $fq1 $fq2"
    echo $cm
    $cm &> logs/trim_galore/$log_file;
    wait
done

trim_galore --core 3 --paired --rrbs -o fastq fastq/SRR11711272_1.fastq.gz fastq/SRR11711272_2.fastq.gz
trim_galore --core 3 --paired --rrbs -o fastq fastq/SRR11711273_1.fastq.gz fastq/SRR11711273_2.fastq.gz


In [9]:
mv -v fastq/*trimming_report.txt logs/trim_galore/

‘fastq/SRR11711272_1.fastq.gz_trimming_report.txt’ -> ‘logs/trim_galore/SRR11711272_1.fastq.gz_trimming_report.txt’
‘fastq/SRR11711272_2.fastq.gz_trimming_report.txt’ -> ‘logs/trim_galore/SRR11711272_2.fastq.gz_trimming_report.txt’
‘fastq/SRR11711273_1.fastq.gz_trimming_report.txt’ -> ‘logs/trim_galore/SRR11711273_1.fastq.gz_trimming_report.txt’
‘fastq/SRR11711273_2.fastq.gz_trimming_report.txt’ -> ‘logs/trim_galore/SRR11711273_2.fastq.gz_trimming_report.txt’


In [10]:
!multiqc logs/trim_galore/ -f -n multiqc-trim


  /// ]8;id=765597;https://multiqc.info\MultiQC]8;;\ 🔍 | v1.12

|           multiqc | Search path : /data_gilbert/home/aarab/Projects/Decitabine-treatment/Bisulfite-seq/logs/trim_galore
|         searching | ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 6/6  xtlogs/trim_galore/SRR11711273.log
|          cutadapt | Found 4 reports
|           multiqc | Compressing plot data
|           multiqc | Deleting    : multiqc-trim.html   (-f was specified)
|           multiqc | Deleting    : multiqc-trim_data   (-f was specified)
|           multiqc | Report      : multiqc-trim.html
|           multiqc | Data        : multiqc-trim_data
|           multiqc | MultiQC complete


## Processing the BS-seq FASTQ files with Bismark

Bismark - 
https://github.com/FelixKrueger/Bismark


It faild maybe because of the genome annotation - [there's no output result #34
](https://github.com/FelixKrueger/Bismark/issues/34)

I've discussed my issues here https://github.com/FelixKrueger/Bismark/issues/475

We solved the issue together! https://github.com/FelixKrueger/Bismark/issues/475#issuecomment-1061147127

### Index genome

__(I) Running bismark_genome_preparation__

In [109]:
# !bismark_genome_preparation ~/genomes/hg38/gencode.v34/

In [112]:
!gzip -d /data_gilbert/home/aarab/genomes/hg38/chromosomes/*.fa.gz

In [ ]:
!bismark_genome_preparation --verbose ~/genomes/hg38/chromosomes/

Path to genome folder specified as: /data_gilbert/home/aarab/genomes/hg38/chromosomes/
Aligner to be used: >> Bowtie 2 << (default)
Writing bisulfite genomes out into a single MFA (multi FastA) file

Bismark Genome Preparation - Step I: Preparing folders

Bisulfite Genome Indexer version v0.23.1 (last modified: 27 Jan 2021)
Created Bisulfite Genome folder /data_gilbert/home/aarab/genomes/hg38/chromosomes/Bisulfite_Genome/
Created Bisulfite Genome folder /data_gilbert/home/aarab/genomes/hg38/chromosomes/Bisulfite_Genome/CT_conversion/
Created Bisulfite Genome folder /data_gilbert/home/aarab/genomes/hg38/chromosomes/Bisulfite_Genome/GA_conversion/

Step I - Prepare genome folders - completed


Bismark Genome Preparation - Step II: Bisulfite converting reference genome

conversions performed:
chromosome	C->T	G->A
chr1	48055043	48111528
chr10	27639505	27719976
chr11	27903257	27981801
chr12	27092804	27182678
chr13	18839192	18933605
chr14	18423758	18559033
chr15	17752941	17825903
chr16	18172

In [35]:
!bismark_genome_preparation --verbose --hisat2 ~/genomes/hg38/chromosomes/

Path to genome folder specified as: /data_gilbert/home/aarab/genomes/hg38/chromosomes/
Aligner to be used: >> HISAT2 <<
Writing bisulfite genomes out into a single MFA (multi FastA) file

Bismark Genome Preparation - Step I: Preparing folders

Bisulfite Genome Indexer version v0.23.1 (last modified: 27 Jan 2021)

A directory called /data_gilbert/home/aarab/genomes/hg38/chromosomes/Bisulfite_Genome/ already exists. Already existing converted sequences and/or already existing Bowtie 2 or HISAT2) indices will be overwritten!


Step I - Prepare genome folders - completed


Bismark Genome Preparation - Step II: Bisulfite converting reference genome

conversions performed:
chromosome	C->T	G->A
chr1	48055043	48111528
chr10	27639505	27719976
chr11	27903257	27981801
chr12	27092804	27182678
chr13	18839192	18933605
chr14	18423758	18559033
chr15	17752941	17825903
chr16	18172742	18299976
chr17	18723944	18851500
chr18	15794455	16061651
chr19	13954580	14061132
chr2	48318180	48450903
chr20	13916133	14

### Analyze samples

__(II) Running bismark__

In [ ]:
%%bash 
genDIR="/data_gilbert/home/aarab/genomes/hg38/chromosomes/"

cd fastq
for fq1 in *_val_1.fq.gz; do 
    fq2=${fq1/_1_val_1.fq/_2_val_2.fq}
    b=`basename $fq1`
    sample=${b/_1_val_1.fq.gz/}
    cm="bismark --gzip --genome $genDIR -1 $fq1 -2 $fq2"
    # --score_min L,0,-0.4
    echo $cm
    $cm &> ${sample}.log;
    wait
done
cd ../

bismark --gzip --genome /data_gilbert/home/aarab/genomes/hg38/chromosomes/ -1 SRR11711272_1_val_1.fq.gz -2 SRR11711272_2_val_2.fq.gz


__(III) Running deduplicate_bismark__
> should not be used for reduced representation libraries such as RRBS, amplicon or target enrichment libraries.

however, 

https://pubmed.ncbi.nlm.nih.gov/33958785/

> Alignments were futher filtered to only include reads that are uniquely aligned, map to an MspI cut site, and do not represent PCR duplicates (taking into account the hexamer sequence as a unique molecular identifier).

In [6]:
!deduplicate_bismark

Please provide one or more Bismark output files for deduplication



This script is supposed to remove alignments to the same position in the genome from the Bismark mapping output
(both single and paired-end SAM/BAM files), which can arise by e.g. excessive PCR amplification. If sequences align
to the same genomic position but on different strands they will be scored individually.

Note that deduplication is not recommended for RRBS-type experiments!

In the default mode, the first alignment to a given position will be used irrespective of its methylation call
(as the alignments are not ordered in any way this is also near enough random).

For single-end alignments only the chromosome, start coordinate and strand of a read will be used for deduplication.

For paired-end alignments the chromosome, strand alignments, and start as well as end coordinate of the entire fragment
will be used for deduplication. >This script expects the Bismark output to be in SAM format<.

*** Please note th

In [13]:
!zcat bismark/SRR11711272_1_val_1.fq.gz | head

@SRR11711272.1 NS500400:843:HVTWWBGXC:1:11101:3311:1050:TCGCTAGA+GGAGAGTA:GGATGGTT+ACTCTN/1
CGGGATATTTGGTTGTTNNAAATAATATG
+
EA6EEEEEEEEEEAEEE##EEEEEEEEEE
@SRR11711272.2 NS500400:843:HVTWWBGXC:1:11101:11560:1050:TCGCTAGA+GGAGAGTA:GGATGGTT+CCCCAN/1
TGGTTGATTGTTTAGTTNNATGTAGGTTA
+
EEAEEEEEEEEEEEAEE##/EEE/EEEEE
@SRR11711272.3 NS500400:843:HVTWWBGXC:1:11101:2495:1050:TCGCTAGA+GGAGAGTA:GGATGGTT+CAACCN/1
CGGTTATTATTTTAGTTNNGATTAGAATTTAT

gzip: stdout: Broken pipe


In [15]:
%%bash 
for bam in bismark/*_bt2_pe.bam; do 
    echo $bam
    deduplicate_bismark --bam $bam
done 

bismark/SRR11711272_1_val_1_bismark_bt2_pe.bam



Neither -s (single-end) nor -p (paired-end) selected for deduplication. Trying to extract this information for each file separately from the @PG line of the SAM/BAM file
Processing single-end Bismark output file(s) (SAM format):
bismark/SRR11711272_1_val_1_bismark_bt2_pe.bam


If there are several alignments to a single position in the genome the first alignment will be chosen. Since the input files are not in any way sorted this is a near-enough random selection of reads.

Checking file >>bismark/SRR11711272_1_val_1_bismark_bt2_pe.bam<< for signs of file truncation...

Trying to determine the type of mapping from the SAM header line
Treating file as paired-end data (extracted from @PG line)




...passed!
Output file is: SRR11711272_1_val_1_bismark_bt2_pe.deduplicated.bam


Total number of alignments analysed in bismark/SRR11711272_1_val_1_bismark_bt2_pe.bam:	24516791
Total number duplicated alignments removed:	3437275 (14.02%)
Duplicated alignments were found at:	2529677 different position(s)

Total count of deduplicated leftover sequences: 21079516 (85.98% of total)



Now testing Bismark result file bismark/SRR11711272_1_val_1_bismark_bt2_pe.bam for positional sorting (which would be bad...)	skipping header line:	@HD	VN:1.0	SO:unsorted
skipping header line:	@SQ	SN:chr1	LN:248956422
skipping header line:	@SQ	SN:chr10	LN:133797422
skipping header line:	@SQ	SN:chr11	LN:135086622
skipping header line:	@SQ	SN:chr12	LN:133275309
skipping header line:	@SQ	SN:chr13	LN:114364328
skipping header line:	@SQ	SN:chr14	LN:107043718
skipping header line:	@SQ	SN:chr15	LN:101991189
skipping header line:	@SQ	SN:chr16	LN:90338345
skipping header line:	@SQ	SN:chr17	LN:83257441
skipping header line:	@SQ	SN:chr18	LN:80373285
skipping header line:	@SQ	SN:chr19	LN:58617616
skipping header line:	@SQ	SN:chr2	LN:242193529
skipping header line:	@SQ	SN:chr20	LN:64444167
skipping header line:	@SQ	SN:chr21	LN:46709983
skipping header line:	@SQ	SN:chr22	LN:50818468
skipping header line:	@SQ	SN:chr3	LN:198295559
skipping header line:	@SQ	SN:chr4	LN:190214555
skipping header line:	@S


Neither -s (single-end) nor -p (paired-end) selected for deduplication. Trying to extract this information for each file separately from the @PG line of the SAM/BAM file
Processing single-end Bismark output file(s) (SAM format):
bismark/SRR11711273_1_val_1_bismark_bt2_pe.bam


If there are several alignments to a single position in the genome the first alignment will be chosen. Since the input files are not in any way sorted this is a near-enough random selection of reads.

Checking file >>bismark/SRR11711273_1_val_1_bismark_bt2_pe.bam<< for signs of file truncation...

Trying to determine the type of mapping from the SAM header line
Treating file as paired-end data (extracted from @PG line)




...passed!
Output file is: SRR11711273_1_val_1_bismark_bt2_pe.deduplicated.bam


Total number of alignments analysed in bismark/SRR11711273_1_val_1_bismark_bt2_pe.bam:	6784977
Total number duplicated alignments removed:	379032 (5.59%)
Duplicated alignments were found at:	309602 different position(s)

Total count of deduplicated leftover sequences: 6405945 (94.41% of total)



Now testing Bismark result file bismark/SRR11711273_1_val_1_bismark_bt2_pe.bam for positional sorting (which would be bad...)	skipping header line:	@HD	VN:1.0	SO:unsorted
skipping header line:	@SQ	SN:chr1	LN:248956422
skipping header line:	@SQ	SN:chr10	LN:133797422
skipping header line:	@SQ	SN:chr11	LN:135086622
skipping header line:	@SQ	SN:chr12	LN:133275309
skipping header line:	@SQ	SN:chr13	LN:114364328
skipping header line:	@SQ	SN:chr14	LN:107043718
skipping header line:	@SQ	SN:chr15	LN:101991189
skipping header line:	@SQ	SN:chr16	LN:90338345
skipping header line:	@SQ	SN:chr17	LN:83257441
skipping header line:	@SQ	SN:chr18	LN:80373285
skipping header line:	@SQ	SN:chr19	LN:58617616
skipping header line:	@SQ	SN:chr2	LN:242193529
skipping header line:	@SQ	SN:chr20	LN:64444167
skipping header line:	@SQ	SN:chr21	LN:46709983
skipping header line:	@SQ	SN:chr22	LN:50818468
skipping header line:	@SQ	SN:chr3	LN:198295559
skipping header line:	@SQ	SN:chr4	LN:190214555
skipping header line:	@S

__(IV) Running bismark_methylation_extractor__
extract context-dependent (CpG/CHG/CHH) methylation

This will produce three methytlation output files:

    CpG_context_test_dataset_bismark_bt2.txt.gz
    CHG_context_test_dataset_bismark_bt2.txt.gz
    CHH_context_test_dataset_bismark_bt2.txt.gz
as well as a bedGraph and a Bismark coverage file. For more on these files and their formats please see below.

In [ ]:
%%bash 
cd bismark/
for bam in *deduplicated.bam; do 
    bismark_methylation_extractor --multicore 10 --gzip --bedGraph $bam
done
cd ../


 *** Bismark methylation extractor version v0.23.1 ***

Trying to determine the type of mapping from the SAM header line of file SRR11711272_1_val_1_bismark_bt2_pe.deduplicated.bam
Treating file(s) as paired-end data (as extracted from @PG line)

Setting option '--no_overlap' since this is (normally) the right thing to do for paired-end data

Summarising Bismark methylation extractor parameters:
Bismark paired-end SAM format specified (default)
Number of cores to be used: 10
Output will be written to the current directory ('/data_gilbert/home/aarab/Projects/Decitabine-treatment/DAC/Bisulfite-seq/bismark')


Summarising bedGraph parameters:
Generating additional output in bedGraph and coverage format
bedGraph format:	<Chromosome> <Start Position> <End Position> <Methylation Percentage>
coverage format:	<Chromosome> <Start Position> <End Position> <Methylation Percentage> <count methylated> <count non-methylated>

Using a cutoff of 1 read(s) to report cytosine positions
Reporting and so

...passed!
Writing result file containing methylation information for C in CpG context from the original top strand to CpG_OT_SRR11711272_1_val_1_bismark_bt2_pe.deduplicated.txt.gz
Writing result file containing methylation information for C in CpG context from the complementary to original top strand to CpG_CTOT_SRR11711272_1_val_1_bismark_bt2_pe.deduplicated.txt.gz
Writing result file containing methylation information for C in CpG context from the complementary to original bottom strand to CpG_CTOB_SRR11711272_1_val_1_bismark_bt2_pe.deduplicated.txt.gz
Writing result file containing methylation information for C in CpG context from the original bottom strand to CpG_OB_SRR11711272_1_val_1_bismark_bt2_pe.deduplicated.txt.gz

Writing result file containing methylation information for C in CHG context from the original top strand to CHG_OT_SRR11711272_1_val_1_bismark_bt2_pe.deduplicated.txt.gz
Writing result file containing methylation information for C in CHG context from the complemen

Now testing Bismark result file >SRR11711272_1_val_1_bismark_bt2_pe.deduplicated.bam< for positional sorting (which would be bad...)	SRR11711272_1_val_1_bismark_bt2_pe.deduplicated_splitting_report.txt.1
SRR11711272_1_val_1_bismark_bt2_pe.deduplicated_splitting_report.txt.2
SRR11711272_1_val_1_bismark_bt2_pe.deduplicated_splitting_report.txt.3
SRR11711272_1_val_1_bismark_bt2_pe.deduplicated_splitting_report.txt.4
SRR11711272_1_val_1_bismark_bt2_pe.deduplicated_splitting_report.txt.5
SRR11711272_1_val_1_bismark_bt2_pe.deduplicated_splitting_report.txt.6
SRR11711272_1_val_1_bismark_bt2_pe.deduplicated_splitting_report.txt.7
SRR11711272_1_val_1_bismark_bt2_pe.deduplicated_splitting_report.txt.8
SRR11711272_1_val_1_bismark_bt2_pe.deduplicated_splitting_report.txt.9
SRR11711272_1_val_1_bismark_bt2_pe.deduplicated_splitting_report.txt.10




Processed 21079516 lines in total
Total number of methylation call strings processed: 42159032

Final Cytosine Methylation Report
Total number of C's analysed:	292164183

Total methylated C's in CpG context:	25968503
Total methylated C's in CHG context:	784307
Total methylated C's in CHH context:	2155710

Total C to T conversions in CpG context:	15713878
Total C to T conversions in CHG context:	65131428
Total C to T conversions in CHH context:	182410357

C methylated in CpG context:	62.3%
C methylated in CHG context:	1.2%
C methylated in CHH context:	1.2%



Merging individual M-bias reports into overall M-bias statistics from these 10 individual files:


SRR11711272_1_val_1_bismark_bt2_pe.deduplicated_splitting_report.txt.1.mbias
SRR11711272_1_val_1_bismark_bt2_pe.deduplicated_splitting_report.txt.2.mbias
SRR11711272_1_val_1_bismark_bt2_pe.deduplicated_splitting_report.txt.3.mbias
SRR11711272_1_val_1_bismark_bt2_pe.deduplicated_splitting_report.txt.4.mbias
SRR11711272_1_val_1_bismark_bt2_pe.deduplicated_splitting_report.txt.5.mbias
SRR11711272_1_val_1_bismark_bt2_pe.deduplicated_splitting_report.txt.6.mbias
SRR11711272_1_val_1_bismark_bt2_pe.deduplicated_splitting_report.txt.7.mbias
SRR11711272_1_val_1_bismark_bt2_pe.deduplicated_splitting_report.txt.8.mbias
SRR11711272_1_val_1_bismark_bt2_pe.deduplicated_splitting_report.txt.9.mbias
SRR11711272_1_val_1_bismark_bt2_pe.deduplicated_splitting_report.txt.10.mbias



Determining maximum read lengths for M-Bias plots
Maximum read length of Read 1: 32
Maximum read length of Read 2: 28

Perl module GD::Graph::lines is not installed, skipping drawing M-bias plots (only writing out M-bias plot table)
Determining maximum read lengths for M-Bias plots
Maximum read length of Read 1: 32
Maximum read length of Read 2: 28

Perl module GD::Graph::lines is not installed, skipping drawing M-bias plots (only writing out M-bias plot table)
Deleting unused files ...

CpG_OT_SRR11711272_1_val_1_bismark_bt2_pe.deduplicated.txt.gz contains data ->	kept
CpG_CTOT_SRR11711272_1_val_1_bismark_bt2_pe.deduplicated.txt.gz was empty ->	deleted
CpG_CTOB_SRR11711272_1_val_1_bismark_bt2_pe.deduplicated.txt.gz was empty ->	deleted
CpG_OB_SRR11711272_1_val_1_bismark_bt2_pe.deduplicated.txt.gz contains data ->	kept
CHG_OT_SRR11711272_1_val_1_bismark_bt2_pe.deduplicated.txt.gz contains data ->	kept
CHG_CTOT_SRR11711272_1_val_1_bismark_bt2_pe.deduplicated.txt.gz was empty ->	deleted


/data_gilbert/home/aarab/Projects/Decitabine-treatment/DAC/Bisulfite-seq/bismark/CpG_OT_SRR11711272_1_val_1_bismark_bt2_pe.deduplicated.txt.gz	/data_gilbert/home/aarab/Projects/Decitabine-treatment/DAC/Bisulfite-seq/bismark/CpG_OB_SRR11711272_1_val_1_bismark_bt2_pe.deduplicated.txt.gz



Writing bedGraph to file: SRR11711272_1_val_1_bismark_bt2_pe.deduplicated.bedGraph.gz
Also writing out a coverage file including counts methylated and unmethylated residues to file: SRR11711272_1_val_1_bismark_bt2_pe.deduplicated.bismark.cov.gz

Now writing methylation information for file >>CpG_OT_SRR11711272_1_val_1_bismark_bt2_pe.deduplicated.txt.gz<< to individual files for each chromosome
Finished writing out individual chromosome files for CpG_OT_SRR11711272_1_val_1_bismark_bt2_pe.deduplicated.txt.gz
Now writing methylation information for file >>CpG_OB_SRR11711272_1_val_1_bismark_bt2_pe.deduplicated.txt.gz<< to individual files for each chromosome


In [25]:
ls bismark/

bismark_summary_report.html
bismark_summary_report.txt
CHG_OB_SRR11711272_1_val_1_bismark_bt2_pe.deduplicated.txt.gz
CHG_OB_SRR11711272_1_val_1_bismark_bt2_pe.txt.gz
CHG_OB_SRR11711273_1_val_1_bismark_bt2_pe.deduplicated.txt.gz
CHG_OB_SRR11711273_1_val_1_bismark_bt2_pe.txt.gz
CHG_OT_SRR11711272_1_val_1_bismark_bt2_pe.deduplicated.txt.gz
CHG_OT_SRR11711272_1_val_1_bismark_bt2_pe.txt.gz
CHG_OT_SRR11711273_1_val_1_bismark_bt2_pe.deduplicated.txt.gz
CHG_OT_SRR11711273_1_val_1_bismark_bt2_pe.txt.gz
CHH_OB_SRR11711272_1_val_1_bismark_bt2_pe.deduplicated.txt.gz
CHH_OB_SRR11711272_1_val_1_bismark_bt2_pe.txt.gz
CHH_OB_SRR11711273_1_val_1_bismark_bt2_pe.deduplicated.txt.gz
CHH_OB_SRR11711273_1_val_1_bismark_bt2_pe.txt.gz
CHH_OT_SRR11711272_1_val_1_bismark_bt2_pe.deduplicated.txt.gz
CHH_OT_SRR11711272_1_val_1_bismark_bt2_pe.txt.gz
CHH_OT_SRR11711273_1_val_1_bismark_bt2_pe.deduplicated.txt.gz
CHH_OT_SRR11711273_1_val_1_bismark_bt2_pe.txt.gz
CpG_OB_SRR11711272_1_val_1_bismark_bt2_pe.deduplicated.tx

<!-- __(V) Running bismark2report__ -->

__(VI) Running bismark2summary__

In [26]:
%%bash 
cd bismark/ 
bismark2summary
cd ../

No Bismark/Bowtie2 single-end BAM files detected
Found Bismark/Bowtie2 paired-end files
No Bismark/HISAT2 single-end BAM files detected
No Bismark/HISAT2 paired-end BAM files detected

Generating Bismark summary report from 2 Bismark BAM file(s)...
>> Reading from Bismark report: SRR11711272_1_val_1_bismark_bt2_PE_report.txt
>> Reading from Bismark report: SRR11711273_1_val_1_bismark_bt2_PE_report.txt

Wrote Bismark project summary to >> bismark_summary_report.html <<



___
for methylkit

In [29]:
%%bash 
for bam in bismark/*deduplicated.bam; do 
    out=${bam/.bam/.srt.bam}
    cm="samtools sort -@ 24 -o $out $bam"
    echo $cm
    $cm; 
    wait
done 

samtools sort -@ 24 -o bismark/SRR11711272_1_val_1_bismark_bt2_pe.deduplicated.srt.bam bismark/SRR11711272_1_val_1_bismark_bt2_pe.deduplicated.bam


[bam_sort_core] merging from 0 files and 24 in-memory blocks...


samtools sort -@ 24 -o bismark/SRR11711273_1_val_1_bismark_bt2_pe.deduplicated.srt.bam bismark/SRR11711273_1_val_1_bismark_bt2_pe.deduplicated.bam


[bam_sort_core] merging from 0 files and 24 in-memory blocks...


# 

In [30]:
!date

Wed Oct 19 13:34:29 PDT 2022
